# ***Question 3 - Best non fossil fuel price in the future***

At first I'm importing the required packages so I can write my code clearly.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from cycler import cycler

I have found a dataset with cost per renewal energy source

This dataset is the one I will use to determine the answer to this topic

I'm importing the dataset below.

In [ ]:
# import data
data_source_link = "https://ourworldindata.org/grapher/levelized-cost-of-energy.csv?v=1&csvType=full&useColumnShortNames=false"

df = pd.read_csv(data_source_link, header = 0, names=['Entity', 'Code', 'Year', 'Bio_cost', 'Geo_cost', 'Off_wind_cost', 'Solar_pv_cost', 'Solar_cc_cost', 'Hydro_cost', 'On_wind_cost'])

Next step is to clean the data by dropping missing values, I noticed that there was only workable data for all the energy sources in the world Entity, for others only solar pv and/or onshore wind data available.

So I'm using only data from the world Entity, which if you think of it is also the best to have an overall insight.

In [ ]:
# Remove rows with NaN values
df_clean = df.dropna()

X = df_clean["Year"].values.reshape(-1, 1)  # Independent variable (years)
models = {}
predictions = {}

future_years = np.array([2025, 2030, 2035]).reshape(-1, 1)

for col in df_clean.columns:
    if col not in ['Year', 'Code', 'Entity'] :
        model = LinearRegression()
        model.fit(X, df_clean[col])
        models[col] = model
        predictions[col] = model.predict(future_years)


In [ ]:
df_clean

To see which energy source is predicted to be the cheapest in 2035, we can compare the predicted costs for that year.

I did this by using a linear regression model to predict the costs for each energy source in 2025, 2030, and 2035.

Because linear regression is a simple and effective method for modeling trends over time.

In [ ]:
# 3. Print predictions

for energy_type, preds in predictions.items():
    print(f"\n### {energy_type.upper()} price predictions")
    for year, price in zip(future_years.flatten(), preds):
        print(f"  {year}: {price:.2f} USD/MWh")

I'm using all of the outcomes above in my graph so i can visually asses which non fossil fuel is most likely to be the cheapest in the future.

In [ ]:
# 4. Plot forecasts

# create a color for each energy type and repeat each color twice
cols = [c for c in df_clean.columns if c not in ['Year', 'Code', 'Entity']]
n = len(cols)
cmap = plt.get_cmap('tab20' if n > 10 else 'tab10')
base_colors = cmap(np.linspace(0, 1, n))
rep_colors = np.repeat(base_colors, 2, axis=0)  # repeat each color so historical+forecast use same color

plt.rcParams['axes.prop_cycle'] = cycler('color', [tuple(c) for c in rep_colors])
plt.figure(figsize=(12, 7))

for col in df_clean.columns:
    if col not in ['Year', 'Code', 'Entity']:
        plt.plot(df_clean["Year"], df_clean[col], marker='o', label=f"{col} (historical)")
        plt.plot(future_years, predictions[col], marker='x', linestyle='--', label=f"{col} (forecast)")

plt.xlabel("Year")
plt.ylabel("Price (USD/MWh)")
plt.title("Forecast of Non-Fossil-Fuel Energy Prices Using Linear Regression")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

As seen in the plot, solar photovoltaic (solar_pv_cost) is predicted to be the cheapest energy source by 2035. both of the solar energy source prices see the steepest decline over the forecast period, indicating significant cost reductions compared to other energy sources.

In reality a price under zero is not likely which is probably caused because the curve for solar and wind seems to be quadratic in nature. This may indicate that linear regression may not be the best fit for predicting a reliable future price.

# ***Conclusion***

Solar seems to be the cheapest non-fossil fuel source in the future.